# 프로젝트: 한영 번역기 만들기

In [1]:
import requests
import tarfile

import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

from konlpy.tag import Mecab

2024-12-20 14:46:18.733883: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Step 1. 데이터 다운로드

In [2]:
# # 다운로드할 파일의 URL
# url = "https://raw.githubusercontent.com/jungyeul/korean-parallel-corpora/master/korean-english-news-v1/korean-english-park.train.tar.gz"

# # 파일을 저장할 경로
# filename = "korean-english-park.train.tar.gz"

# # 파일 다운로드
# response = requests.get(url)
# with open(filename, 'wb') as file:
#     file.write(response.content)

# print(f"{filename} 파일이 성공적으로 다운로드되었습니다.")

# # 파일 경로
# file_name = "korean-english-park.train.tar.gz"

# # 압축 해제 및 추출
# with tarfile.open(file_name, "r:gz") as tar:
#     tar.extractall()  # 현재 디렉토리에 파일을 추출
#     print(f"{file_name} 파일이 성공적으로 추출되었습니다.")

In [3]:
kor_path = 'data/macine_translate/korean-english-park.train.ko'
eng_path = 'data/macine_translate/korean-english-park.train.en'

with open(kor_path, 'r') as f:
    kor = f.read().splitlines()
    
with open(eng_path, 'r') as f:
    eng = f.read().splitlines()
    
print(f"한국어 데이터 개수: {len(kor)}")
print(f"영어 데이터 개수: {len(eng)}")

한국어 데이터 개수: 94123
영어 데이터 개수: 94123


## Step 2. 데이터 정제 

set 데이터형이 중복을 허용하지 않는다는 것을 활용해 중복된 데이터를 제거하도록 합니다. 데이터의 병렬 쌍이 흐트러지지 않게 주의하세요! 중복을 제거한 데이터를 cleaned_corpus 에 저장합니다.

In [4]:
# set()을 사용하여 중복을 제거한 뒤 다시 리스트로 변환

cleaned_corpus = list(set(zip(kor, eng)))

print(f"전체 문장 쌍의 개수: {len(cleaned_corpus)}")

# 한국어 list
cleaned_kor = []
# 영어 list
cleaned_eng = []

for kor, eng in cleaned_corpus:
    cleaned_kor.append(kor)
    cleaned_eng.append(eng)
    
print(f"한국어: {cleaned_kor[100]}")
print(f"영어: {cleaned_eng[100]}")


전체 문장 쌍의 개수: 78968
한국어: 캄포트 지역의 누온 사리 경찰 차장은 시엠 레아프로부터 남쪽으로 비행하던 러시아제 AN-24 항공기가 산에 충돌하며 생존자가 남지 않았다고 언급했다고, AFP 통신이 보도했다.
영어: Nuon Sary, deputy police chief of Kampot province, said that the Russian-made AN-24 flying from Siem Reap to the south seemed to have hit the mountain, leaving no survivors, according to AFP.


데이터 확인

In [5]:
print(cleaned_kor[10000])
print(cleaned_eng[10000])

이 자리에서 영화는 20분만 상영됐지만 반응이 좋아 올해 여름 시장에서 큰 호응이 예상된다.
From the 20 minutes of footage they showed, the film looks likely to meet expectations as one of summer's hottest tickets.


In [6]:
print(cleaned_kor[9709])
print(cleaned_eng[9709])

무게가 더 나가면 연료도 많이 소비된다.
More weight means more fuel.


In [7]:
print(cleaned_kor[1000:1005])
print(cleaned_eng[1000:1005])

['이 일은 계속됩니다.', '영국 당국이 이 개정안에따라 기소 전 테러 용의자를 6주간 구금할 수 있게 된다.', '“우리는 백 년에 한번 일어날만한 신용 쓰나미의 한 가운데에 있습니다.”', '미얀마 정부는 통금령을 완화해 13일부터 밤 11시부터 새벽 3시까지로 4시간으로 줄었다.', '토지 가격이 신축 아파트 가격의 70%에 이르기 때문에, 건축 비용만 부담하게 한다면 집값을 내리는 데 도움이 될 것이지만, 정책의 실행 가능성은 아직 알 수 없다.']
['This work continues.', 'Britain¡¯s House of Commons on Wednesday narrowly approved a counterterrorism bill that allows authorities to hold terrorism suspects without charge for up to six weeks.', '“We are in the midst of a once-in-a-century credit tsunami.', 'Authorities in the country relaxed a nighttime curfew to cover four hours from 11 p.m. to 3 a.m starting Saturday night.', 'As land costs account for up to 70 percent of new apartment prices, having people pay just the construction costs would help cut home prices, although the feasibility of the policy remains to be determined.']


앞서 정의한 preprocessing() 함수는 한글에서는 동작하지 않습니다. 한글에 적용할 수 있는 정규식을 추가하여 함수를 재정의하세요!

In [8]:
# preprocess() 함수를 정의

def kor_prerocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^가-힣?.!,]+", " ", sentence)
    
    sentence = sentence.strip()

    return sentence


def eng_prerocess_sentence(sentence, s_token=False, e_token=False):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    
    sentence = sentence.strip()
    
    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
        
    return sentence

In [9]:
# preprocess() 함수를 사용하여 데이터를 정제

kor_corpus = []
eng_corpus = []

for kor, eng in zip(cleaned_kor, cleaned_eng):
    # 토큰 길이가 40이하인 데이터 대상으로 정제
    if len(kor) <= 40:
        kor_corpus.append(kor_prerocess_sentence(kor))
        eng_corpus.append(eng_prerocess_sentence(eng, s_token=True, e_token=True))
    
print(f"한국어: {kor_corpus[100]}")
print(f"영어: {eng_corpus[100]}")

print(len(kor_corpus))
print(len(eng_corpus))


한국어: 샤론 총리는 파리 방문을 계획 중이었으나 정해진 일정은 없었다 .
영어: <start> Sharon was considering a trip to Paris , but no date had been set . <end>
15931
15931


## Step 3. 데이터 토큰화

앞서 정의한 tokenize() 함수를 사용해 데이터를 텐서로 변환하고 각각의 tokenizer를 얻으세요! 단어의 수는 실험을 통해 적당한 값을 맞춰주도록 합니다! (최소 10,000 이상!)

❗ 주의: 난이도에 비해 데이터가 많지 않아 훈련 데이터와 검증 데이터를 따로 나누지는 않습니다.

In [10]:
def kor_tokenizer(corpus):
    mecab = Mecab()
    # 형태소 분석기를 활용한 토큰화
    tokenized_corpus = [mecab.morphs(sentence) for sentence in corpus]
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(tokenized_corpus)
    
    tensor = tokenizer.texts_to_sequences(tokenized_corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    return tensor, tokenizer

def eng_tokenizer(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    return tensor, tokenizer

In [11]:
# tokenize
kor_tensor, kor_tokenizer = kor_tokenizer(kor_corpus)
eng_tensor, eng_tokenizer = eng_tokenizer(eng_corpus)

print(kor_tensor.shape)
print(eng_tensor.shape)

(15931, 25)
(15931, 71)


In [12]:
print(len(kor_tokenizer.index_word))
print(len(eng_tokenizer.index_word))

15545
16834


## Step 4. 모델 설계

한국어를 영어로 잘 번역해 줄 멋진 Attention 기반 Seq2seq 모델을 설계하세요! 앞서 만든 모델에 Dropout 모듈을 추가하면 성능이 더 좋아집니다! Embedding Size와 Hidden Size는 실험을 통해 적당한 값을 맞춰 주도록 합니다!

In [13]:
# attention - BahdanauAttention
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units, dropout_rate=0.5):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
    
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)

        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        attn = tf.nn.softmax(score, axis=1)
        attn = self.dropout(attn) # dropout
        
        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)

        return context_vec, attn

In [14]:
# encoder
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, dropout_rate=0.5):
        super(Encoder, self).__init__()
        
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units,
                                       return_sequences=True)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.dropout(out) # dropout
        out = self.gru(out)
        out = self.dropout(out) # dropout
         
        return out

In [15]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, dropout_rate=0.5):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units,
                                       return_sequences=True,
                                       return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.attention = BahdanauAttention(self.dec_units)
        

    def call(self, x, h_dec, enc_out):
        """
        x: Decoder의 input token [batch x 1]
        h_dec: Decoder의 이전 hidden state [batch x units]
        enc_out: Encoder의 output sequence [batch x length x units]
        """
        context_vec, attn = self.attention(enc_out, h_dec)

        out = self.embedding(x)
        out = self.dropout(out)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)
        
        out, h_dec = self.gru(out)
        out = self.dropout(out) # dropout
        out = tf.reshape(out, (-1, out.shape[2])) # [batch x length x units] -> [batch x units]
        out = self.fc(out)
        out = self.dropout(out) # dropout

        return out, h_dec, attn

In [16]:
BATCH_SIZE     = 32 # gpu 터져서 줄여서 진행
SRC_VOCAB_SIZE = len(kor_tokenizer.index_word) + 1
TGT_VOCAB_SIZE = len(eng_tokenizer.index_word) + 1

units         = 1024 # gpu 터져서 줄여서 진행
embedding_dim = 256 # gpu 터져서 줄여서 진행

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units) 

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                     sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

2024-12-20 12:38:52.660429: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9621 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:19:00.0, compute capability: 7.5
2024-12-20 12:38:52.660990: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2024-12-20 12:38:52.661532: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9621 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:67:00.0, compute capability: 7.5
2024-12-20 12:38:52.662042: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 9604 MB memory:  -> device: 3, name: NVIDIA GeForce RTX

Encoder Output: (32, 30, 512)
Decoder Output: (32, 16835)
Decoder Hidden State: (32, 512)
Attention: (32, 30, 1)


## Step 5. 훈련하기

In [17]:
# optimizer & loss
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

In [18]:
@tf.function # 훈련을 위한 함수를 컴파일
def train_step(src, tgt, encoder, decoder, optimizer, dec_tok):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape: # tf.GradientTape()는 가중치 갱신을 위한 Gradient를 자동 미분으로 계산
        enc_out = encoder(src) # Encoder에 소스 문장을 전달하면 컨텍스트 벡터인 enc_out을 리턴
        h_dec = enc_out[:, -1] # Encoder의 마지막 time step의 hidden state를 Decoder의 첫번째 hidden state로 사용
        
        dec_src = tf.expand_dims([dec_tok.word_index['<start>']] * bsz, 1) # Decoder에 입력으로 전달할 <start> 토큰 생성

        for t in range(1, tgt.shape[1]): # 타겟 문장의 길이만큼 반복
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out) # enc_out, hidden state를 기반으로 다음 단어(t시점의 입력)를 예측
 
            loss += loss_function(tgt[:, t], pred) # 타겟 단어와 예측 단어로 손실을 계산
            dec_src = tf.expand_dims(tgt[:, t], 1) # t 시점의 타겟 문장을 다음 시점의 Decoder 입력으로 사용
        
    batch_loss = (loss / int(tgt.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

In [19]:
def translate(sentence, encoder, decoder, kor_tokenizer, eng_tokenizer, max_length=30):
    sentence = kor_prerocess_sentence(sentence)  # 전처리
    mecab = Mecab()
    tokenized_sentence = mecab.morphs(sentence)  
    tensor = kor_tokenizer.texts_to_sequences([tokenized_sentence])  
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  # 패딩

    enc_out = encoder(tensor)
    h_dec = enc_out[:, -1]  # Encoder의 마지막 hidden state

    # Decoder에 <start> 토큰 입력
    dec_src = tf.expand_dims([eng_tokenizer.word_index['<start>']], 1)
    result = []  # 번역 결과 저장

    for i in range(max_length):
        pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)
        pred_id = tf.argmax(pred, axis=-1).numpy().flatten()[0]

        if eng_tokenizer.index_word[pred_id] == '<end>':
            break

        result.append(eng_tokenizer.index_word[pred_id])

        dec_src = tf.expand_dims([pred_id], 1) # 다음 시점의 Decoder 입력으로 사용

    return ' '.join(result)

In [20]:
from tqdm import tqdm  # tqdm
import random

EPOCHS = 10
src_list = ['오바마는 대통령이다.', '시민들은 도시 속에 산다.', '커피는 필요 없다.', '일곱 명의 사망자가 발생했다.']

for epoch in range(EPOCHS):
    total_loss = 0
    
    # 배치 인덱스 셔플
    idx_list = list(range(0, kor_tensor.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)  # tqdm으로 진행 상태 표시

    for (batch, idx) in enumerate(t):
        # 배치 데이터 추출 및 train_step 호출
        batch_loss = train_step(
            kor_tensor[idx:idx+BATCH_SIZE],
            eng_tensor[idx:idx+BATCH_SIZE],
            encoder,
            decoder,
            optimizer,
            eng_tokenizer
        )
        total_loss += batch_loss

        # tqdm 상태 업데이트
        t.set_description_str('Epoch %2d' % (epoch + 1))  # Epoch 정보
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))  # 평균 Loss

    # Epoch 종료 후 번역 결과 출력
    print(f"\nEpoch {epoch + 1} Loss: {total_loss / (batch + 1):.4f}")
    print("\n=== Epoch 번역 결과 ===")
    for sentence in src_list:
        translation = translate(sentence, encoder, decoder, kor_tokenizer, eng_tokenizer)
        print(f"소스 문장: {sentence}")
        print(f"번역 문장: {translation}")
    print("=========================\n")

  0%|          | 0/498 [00:00<?, ?it/s]2024-12-20 12:39:56.832195: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb5f8abd450 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-20 12:39:56.832477: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-12-20 12:39:56.832485: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-12-20 12:39:56.832491: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-12-20 12:39:56.832496: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (3): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-12-20 12:39:56.837344: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env 


Epoch 1 Loss: 1.4755

=== Epoch 번역 결과 ===
소스 문장: 오바마는 대통령이다.
번역 문장: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
소스 문장: 시민들은 도시 속에 산다.
번역 문장: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
소스 문장: 커피는 필요 없다.
번역 문장: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
소스 문장: 일곱 명의 사망자가 발생했다.
번역 문장: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the



Epoch  2: 100%|██████████| 498/498 [01:31<00:00,  5.43it/s, Loss 1.4446]



Epoch 2 Loss: 1.4446

=== Epoch 번역 결과 ===
소스 문장: 오바마는 대통령이다.
번역 문장: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
소스 문장: 시민들은 도시 속에 산다.
번역 문장: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
소스 문장: 커피는 필요 없다.
번역 문장: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
소스 문장: 일곱 명의 사망자가 발생했다.
번역 문장: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the



Epoch  3: 100%|██████████| 498/498 [01:31<00:00,  5.44it/s, Loss 1.4447]


...


Epoch 8 Loss: 1.4456

=== Epoch 번역 결과 ===
소스 문장: 오바마는 대통령이다.
번역 문장: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
소스 문장: 시민들은 도시 속에 산다.
번역 문장: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
소스 문장: 커피는 필요 없다.
번역 문장: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
소스 문장: 일곱 명의 사망자가 발생했다.
번역 문장: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .



Epoch  9: 100%|██████████| 498/498 [01:31<00:00,  5.45it/s, Loss 1.4455]



Epoch 9 Loss: 1.4455

=== Epoch 번역 결과 ===
소스 문장: 오바마는 대통령이다.
번역 문장: 
소스 문장: 시민들은 도시 속에 산다.
번역 문장: 
소스 문장: 커피는 필요 없다.
번역 문장: 
소스 문장: 일곱 명의 사망자가 발생했다.
번역 문장: 



Epoch 10: 100%|██████████| 498/498 [01:31<00:00,  5.44it/s, Loss 1.4233]



Epoch 10 Loss: 1.4233

=== Epoch 번역 결과 ===
소스 문장: 오바마는 대통령이다.
번역 문장: 
소스 문장: 시민들은 도시 속에 산다.
번역 문장: 
소스 문장: 커피는 필요 없다.
번역 문장: 
소스 문장: 일곱 명의 사망자가 발생했다.
번역 문장: 



## 회고


- _v1 파일의 경우, 전처리 방식이 길이 40이하로 자른후 토큰화를 진행하여 약 15,000개의 학습 데이터로 진행,  gpu가 계속 터져서 batch_size와 embedding_size를 줄여서 학습을 진행하였지만 성능이 나오지조차 않는..
- _v2 파일의 경우, 전치리 방식을 각 언어에 맞게 토큰화 진행후 토큰 길이 40개 이하로 약 65,000개의 학습데이터 진행, 실행 도중 오류로 현재 학습 중, 추후에 업로드 예정이며 이게 현재까지 성능이 그나마 제일 좋아 보인다.
- _v3 파일의 경우, 전처리 방식은 _v2와 같고 gpu가 터져서 _v1파일처럼 파라미터를 수정후에 학습을 진행하였지만 결과가 좋지 못하다. 이게 번역기인가?....
    내가 썻던 구글 번역기에 대한 안 좋은 기억이 떠오르는거 같다.
- 아쉬운 점
  - Loung attention을 접목해서 코드 작성하고 싶었는데 코드는 짜봤는데 시간이 없어서 못 돌려봤다, 나중에 따로 업로드 해야겠다.
  - attention을 적용한 seq2seq 모델을 배워보고 attention에 대해서 공부할 수 있었다. 수식을 기반으로 이해를 했지만 transformer에도 쓰이는 중요한 메커니즘이니까 주말을 이용해서 Bahdanau_Attention 논문을 정리해봐야겠다. 그래도 최근에 한 프로젝트들이 재밌긴해서 다행이다